In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Buy, sell or do nothing?
The problem at hand is take action or don't take action. We can convert this problem to a more common problem by exploiting `feature_0` as this is belived to represent buy/sell orders. 
Buy orders remain the same, while we can convert sell orders with positive response to orders with negative responses and visa versa.
e.g. `feature_0=-1` and `resp=0.1` <-> `feature_0=1` and `resp=-0.1`.

Assume `feature_0=1` <-> `buy` (workes either way), then the action is taken if `resp_hat*feature_0 > th`.

In [ ]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import MinMaxScaler

from tqdm import tqdm
from random import choices

import kerastuner as kt

## PurgedGroupTimeSeriesSplit
Click the code button to see. 

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

### Loading the training data

In [ ]:
TRAINING = False
FINETUNE = False
FOLDS = 5
SEED = 42

train = pd.read_csv('../input/jane-street-market-prediction/train.csv')
train = train.query('weight > 0').reset_index(drop = True)

train = train.astype({c: np.float32 for c in train.select_dtypes(include='float64').columns}) #limit memory use
train.fillna(train.median(),inplace=True)

features = [c for c in train.columns if 'feature' in c][1:]

resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

weights = train['weight'].values
resps = train['resp'].values
dates = train['date'].values


In [ ]:
train[resp_cols] = train[resp_cols].values * np.expand_dims(train['feature_0'].values,axis=1)

In [ ]:
%%time
#Neutralization
from sklearn.linear_model import LinearRegression
lrs = []
for resp in resp_cols:
    lr = LinearRegression()
    lr.fit(train[features].values,train[resp].values)
    train[resp] = train[resp].values - lr.predict(train[features].values)
    lrs.append(lr)

In [ ]:
X = train[features].values
y = train[resp_cols].values

In [ ]:
f_mean = np.mean(X,axis=0)

### Creating the MLP. 

In [ ]:
class CustomMeanSquaredError(tf.keras.losses.Loss):
    #https://www.kaggle.com/anlgrbz/approx-utility-score-for-regress-weighted-traing/
    def call(self, y_true, y_pred):
        lmbda = 0.2
        y_pred = tf.convert_to_tensor(y_pred)
        y_true = tf.cast(y_true, y_pred.dtype)

        residual = y_true - y_pred

        result = tf.where((y_true * y_pred) >= 0, residual**2, residual**2 - y_true*y_pred*lmbda)

        return tf.reduce_mean(result, axis=-1)

In [ ]:
def create_model(input_dim,output_dim):
    inputs = Input(input_dim)
    x = inputs
    for i in range(3):
        x = BatchNormalization()(x)
        x = Dense(64)(x)
        x = Lambda(tf.keras.activations.swish)(x)
        x = Dropout(0.2)(x)
    x = Dense(output_dim,activation='linear')(x)
    model = Model(inputs=inputs,outputs=x)
    model.compile(optimizer=Adam(0.001),loss=CustomMeanSquaredError())
    return model

In [ ]:
def utility_score_bincount(date, weight, resp, action):
    count_i = len(np.unique(date))
    Pi = np.bincount(date, weight * resp * action)
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / count_i)
    u = np.clip(t, 0, 6) * np.sum(Pi)
    return u

In [ ]:
# From https://medium.com/@micwurm/using-tensorflow-lite-to-speed-up-predictions-a3954886eb98

class LiteModel:
    
    @classmethod
    def from_file(cls, model_path):
        return LiteModel(tf.lite.Interpreter(model_path=model_path))
    
    @classmethod
    def from_keras_model(cls, kmodel):
        converter = tf.lite.TFLiteConverter.from_keras_model(kmodel)
        tflite_model = converter.convert()
        return LiteModel(tf.lite.Interpreter(model_content=tflite_model))
    
    def __init__(self, interpreter):
        self.interpreter = interpreter
        self.interpreter.allocate_tensors()
        input_det = self.interpreter.get_input_details()[0]
        output_det = self.interpreter.get_output_details()[0]
        self.input_index = input_det["index"]
        self.output_index = output_det["index"]
        self.input_shape = input_det["shape"]
        self.output_shape = output_det["shape"]
        self.input_dtype = input_det["dtype"]
        self.output_dtype = output_det["dtype"]
        
    def predict(self, inp):
        inp = inp.astype(self.input_dtype)
        count = inp.shape[0]
        out = np.zeros((count, self.output_shape[1]), dtype=self.output_dtype)
        for i in range(count):
            self.interpreter.set_tensor(self.input_index, inp[i:i+1])
            self.interpreter.invoke()
            out[i] = self.interpreter.get_tensor(self.output_index)[0]
        return out
    
    def predict_single(self, inp):
        """ Like predict(), but only for a single record. The input data can be a Python list. """
        inp = np.array([inp], dtype=self.input_dtype)
        self.interpreter.set_tensor(self.input_index, inp)
        self.interpreter.invoke()
        out = self.interpreter.get_tensor(self.output_index)
        return out[0]

In [ ]:
def optimal_th(oof):
    oof /= train['weight'].values
    oof *= train['feature_0'].values
    pmin = oof.min()
    pmax = oof.max()
    oof = oof*f0s
    r = pmax - pmin
    best_us = 0
    best_th = 0
    for th in np.arange(pmin,pmax,r/1000):
        us = utility_score_bincount(dates, weights, resps, np.where(oof>th,1,0))
        if us > best_us:
            best_us = us
            best_th = th
    return best_th
        

In [ ]:
%%time
val_metric = 'val_loss'
val_direction = 'min'
EPOCHS = 1000
BATCH_SIZE=4096


FOLDS = 5
SEED = 42

if TRAINING:
    ths = []
    for SEED in range(3):
        oof = np.zeros(y.shape)
        
        gkf = PurgedGroupTimeSeriesSplit(n_splits = FOLDS, group_gap=20)
        for fold, (train_indices, test_indices) in enumerate(gkf.split(y, groups=dates)):
            model = create_model(X.shape[-1],y.shape[-1])
            X_train, X_test = X[train_indices], X[test_indices]
            y_train, y_test = y[train_indices], y[test_indices]
            model.fit(X_train,y_train,
                      validation_data=(X_test,y_test),
                      epochs=EPOCHS,
                      batch_size=BATCH_SIZE,
                      callbacks=[EarlyStopping(val_metric,mode=val_direction,patience=10,restore_best_weights=True)])
            model.save_weights(f'./model_{SEED}_{fold}.tf')
            oof[test_indices] += model.predict(X_test)
            model.fit(X_test,y_test,
                      epochs=5,
                      batch_size=BATCH_SIZE)
            model.save_weights(f'./model_{SEED}_{fold}_finetune.tf')
else:
    models = []
    for SEED in range(3):
        for f in range(FOLDS):
            model = create_model(X.shape[-1],y.shape[-1])
            if FINETUNE:
                model.load_weights(f'../input/bshregressionjs/model_{SEED}_{f}_finetune.tf')
            else:
                model.load_weights(f'../input/bshregressionjs/model_{SEED}_{f}.tf')
            model = LiteModel.from_keras_model(model)
            models.append(model)
    

## Submission

In [ ]:
predictions = []

In [ ]:
if not TRAINING:
    import janestreet
    janestreet.competition.make_env.__called__ = False
    env = janestreet.make_env()
    th = 0.001
    for (test_df, pred_df) in tqdm(env.iter_test()):
        w = test_df['weight'].item()
        if w > 0:
            f0 = test_df['feature_0'].item()
            x_tt = test_df.loc[:, features].values
            if np.isnan(x_tt.sum()):
                x_tt = np.nan_to_num(x_tt) + np.isnan(x_tt) * f_mean
            pred = np.mean([model.predict(x_tt) for model in models],axis=0).ravel()
            
            for i,lr in enumerate(lrs):
                pred[i] = pred[i] + lr.predict(x_tt)
            
            pred = np.mean(pred)
            predictions.append(pred*f0)
            pred = np.where(pred * f0 > th, 1, 0).astype(int)
            pred_df.action = pred
        else:
            pred_df.action = 0
        env.predict(pred_df)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(predictions,bins=100)